In [1]:
from bin.config import *
from bin.consumer import Consumer

In [2]:
spark_session: SparkSession = (
    SparkSession.builder.appName("Consumer data for Visualizing")  # type: ignore
    .config("spark.jars.packages", ",".join(SPARK_PACKAGES))
    .getOrCreate()
)

In [5]:
capture_con: Consumer = Consumer(
    topic=CAPTURE_TOPIC,
    schema_list=CAPTURE_SCHEMA_LIST,
    spark_session=spark_session,
)

segment_con: Consumer = Consumer(
    topic=SEGMENT_TOPIC,
    schema_list=SEGMENT_SCHEMA_LIST,
    spark_session=spark_session,
)

In [6]:
segment_table = segment_con.get_history_df()

In [7]:
BATCH_NAME_FILE: str = "batch_folder/batch.csv"
CACHE_NAME_FILE: str = "batch_folder/cache.csv"

In [8]:
def store_dataframe_to_csv(df: DataFrame, path: str, **kwargs) -> None:
    return df.toPandas().to_csv(path, **kwargs)

In [9]:
def handle_batch_event_table(event_df: DataFrame, epoch_id: int) -> None:
    result: DataFrame = event_df.join(segment_table, on="SegmentID", how="inner")
    store_dataframe_to_csv(result, BATCH_NAME_FILE, header=True, index=False)
    
    result = (
        result.groupBy("Boro", "Timestamp")
        .agg(F.sum(F.col("Vol")), F.count(F.col("Vol")))
        .withColumn("Avg(Vol)", F.col("Sum(Vol)") / F.col("Count(Vol)"))
        .drop("Sum(Vol)", "Count(Vol)")
    )

    store_dataframe_to_csv(
        result, CACHE_NAME_FILE, header=False, index=False, mode="a"
    )

In [ ]:
streaming_df: DataFrame = capture_con.get_streaming_df()
streaming_query: StreamingQuery = capture_con.handle_batch_streaming_with_callable(
    streaming_df, handle_batch_event_table
)

print("Streaming query is running...")

try:
    streaming_query.awaitTermination()
except KeyboardInterrupt:
    print("Stopping streaming query...")
    streaming_query.stop()
    print("Streaming query stopped.")

2017-06-20 03:15:00
Streaming query is running...


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\Users\le197\anaconda3\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\le197\anaconda3\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\le197\anaconda3\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\le197\anaconda3\Lib\site-packages\py4j\clientserver.py", line 538, in send_command
    logger.info("Error while receiving.", exc_info=True)
  File "

Stopping streaming query...
